<a href="https://colab.research.google.com/github/gadgil-group/School_Pb_water/blob/main/NY_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
from sklearn.neighbors import DistanceMetric

import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
#import scikitplot as skplt
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.utils import resample
from random import seed
from random import randint
from sklearn.metrics import roc_curve, auc, precision_recall_curve, plot_precision_recall_curve, average_precision_score
from numpy import argmax
from sklearn.metrics import precision_recall_curve, average_precision_score, auc
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import ShuffleSplit
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
import os
import seaborn as sns
from random import sample
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from IPython.display import Image
from pydotplus import graph_from_dot_data
from sklearn.utils import shuffle
from sklearn.base import clone
from sklearn.neural_network import MLPClassifier
drive.mount('/content/drive')
!pip install censusdata
import censusdata


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     |████████████████████████████████| 26.6 MB 1.3 MB/s 
  Created wheel for censusdata: filename=CensusData-1.15-py3-none-any.whl size=28205534 sha256=68790bf407900eadf3583e3285d87f8df40f11a320e94d3e26e215c869c796bc
  Stored in directory: /root/.cache/pip/wheels/17/11/8c/933901298f486bd414f2ab1a62a114085f7d7a19dcbda2dd08
Successfully built censusdata


In [ ]:
Pb_data = pd.read_csv('/content/drive/My Drive/data/NY dataset 2016.csv')
state_ny = scrape_state_data('36')

In [ ]:

Pb_data_merged = Pb_data.merge(right=state_ny, left_on='zip code', right_index=True)

In [ ]:
Pb_data_merged.to_csv('/content/drive/My Drive/data/NY dataset full.csv', index=False)

In [ ]:
Pb_data['zip code'] = Pb_data['Location'].str.extract('(\d{5})')
Pb_data['lat'] = Pb_data['Location'].str.extract('\((\d+.\d+)')
Pb_data['lon'] = Pb_data['Location'].str.extract('([{\-\d+\.\-}]+)\)')



In [ ]:
def scrape_state_data(state_code):
  relevant_tables=[]
# Poverty by age:
  relevant_tables+=['B17020_'+str(i).zfill(3)+'E' for i  in range(1, 18)]
# Voting-age Population by Educational Attainment
  relevant_tables+=['B17020_'+str(i).zfill(3)+'E' for i in range(1, 9)]
# Voting age by poverty
  relevant_tables+=['B29003_'+str(i).zfill(3)+'E' for i in range(1, 4)]
# Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars)
  relevant_tables+=['B29004_001E']
#Sex by age
  relevant_tables+=['B01001_026E', 'B01001_002E']
#Race
  relevant_tables+=['B02001_'+str(i).zfill(3)+'E' for i in range(1, 11)]
#POVERTY STATUS IN THE PAST 12 MONTHS BY AGE BY LANGUAGE SPOKEN AT HOME FOR THE POPULATION 5 YEARS AND OVER	
  relevant_tables+=['B16009_'+str(i).zfill(3)+'E' for i in range(1, 28)]
#POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY HOUSEHOLD TYPE BY TENURE
  relevant_tables+=['B17019_'+str(i).zfill(3)+'E' for i in range(1, 24)]
#POVERTY STATUS OF INDIVIDUALS IN THE PAST 12 MONTHS BY LIVING ARRANGEMENT
  relevant_tables+=['B17021_'+str(i).zfill(3)+'E' for i in range(1, 36)]
#MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) BY VETERAN STATUS BY SEX FOR THE CIVILIAN POPULATION 18 YEARS AND OVER WITH INCOME
  relevant_tables+=['B21004_'+str(i).zfill(3)+'E' for i in range(1, 8)]
#SEX BY AGE BY EMPLOYMENT STATUS FOR THE POPULATION 16 YEARS AND OVER
  relevant_tables+=['B21004_'+str(i).zfill(3)+'E' for i in range(1, 8)]
#POVERTY STATUS IN THE PAST 12 MONTHS OF INDIVIDUALS BY SEX BY EMPLOYMENT STATUS
  relevant_tables+=['B17005_'+str(i).zfill(3)+'E' for i in range(1, 17)]
#Gini index
  relevant_tables+=['B19083_001E']
#FAMILY INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)
  relevant_tables+=['B19101_'+str(i).zfill(3)+'E' for i in range(1, 18)]
  state_data = censusdata.download('acs/acs5', 2019, censusdata.censusgeo([('state', state_code), ('zip code tabulation area', '*')]), relevant_tables)
  new_indices = []
  for index in state_data.index.tolist():
          new_index = index.geo[1][1] 
          new_indices.append(new_index)
  state_data.index = new_indices
  return state_data